In [56]:
# Import Statements
import pandas as pd
import spacy
from sklearn.linear_model import SGDClassifier, LogisticRegression, RidgeClassifier
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [57]:
nlp = spacy.load("en_core_web_lg")
# nlp.Defaults.stop_words.union({'whiskey', 'canadian', 'scotch', 'bourbon', 'barrel', 'mash', 'corn', 'barley', 'peat'})

In [58]:
def tokenize(string):
    tokens = []
    for doc in nlp.pipe(string.lower().split()):
        for token in doc:
            if not token.is_stop and not token.is_punct and token.pos_ != 'PRON':
                tokens.append(token.lemma_)
    return " ".join(tokens)

In [59]:
# def vectorizor(strings):
#     return [nlp(string).vector for string in strings]
    # return nlp(strings).vector

In [60]:
# string = 'I am a simple string. What do I mean? It is a mystery'
# tokens = tokenize(string)
# vectorizor(tokens)

In [61]:
# Get data.

train = pd.read_csv('train.csv')
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.5,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.9,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.0,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.5,1.0
4,6,Davin de Kergommeaux,"After 40 years in barrels, the trademark Canad...",199.0,96,45.0,NaN


In [62]:
train1 = train[~train['category'].isna()]

In [63]:
y = train1['category']
X_train = train1['description'].apply(tokenize)
# vectors = vectorizor(train1['tokens'])
# X_train = pd.DataFrame({'vectors': vectors})

In [64]:
test = pd.read_csv('test.csv')
X_test = test['description'].apply(tokenize)
# test_vectors = vectorizor(test['tokens'])
# X_test = pd.DataFrame({'vectors': test_vectors})

In [73]:
sgd = RidgeClassifier()
vect = TfidfVectorizer(ngram_range=(1, 2),
                       max_features=100)

svd = TruncatedSVD(n_components=50,
                   algorithm='randomized',
                   n_iter=100)

NameError: name 'RidgeClassifier' is not defined

In [74]:
# LSI

lsi = Pipeline([('vect', vect), ('svd', svd)])

In [75]:
# Pipe

pipe = Pipeline([('lsi', lsi), ('clf', sgd)])


In [76]:
# Fit

pipe.fit(X_train, y)
pred = pipe.predict(X_test)

In [77]:
# model = LogisticRegression()
# model.fit(vectors, y)
# pred = model.predict(test_vectors)

In [78]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,category
0,955,1
1,3532,3
2,1390,2
3,1024,4
4,1902,2


In [79]:
submission['category'] = pred.astype(int)
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1


In [80]:
submission.to_csv('submission.csv', index=False)